In [1]:
from keras.applications import VGG19
vgg_conv = VGG19(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

Using TensorFlow backend.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.



In [2]:
from keras.preprocessing.image import ImageDataGenerator

In [3]:
for layer in vgg_conv.layers[:-2]:
    layer.trainable = False
 
for layer in vgg_conv.layers:
    print(layer, layer.trainable)

<keras.engine.input_layer.InputLayer object at 0x7fe2c4fce650> False
<keras.layers.convolutional.Conv2D object at 0x7fe21d26d550> False
<keras.layers.convolutional.Conv2D object at 0x7fe21d26d890> False
<keras.layers.pooling.MaxPooling2D object at 0x7fe21d227d10> False
<keras.layers.convolutional.Conv2D object at 0x7fe21c9ca3d0> False
<keras.layers.convolutional.Conv2D object at 0x7fe21c9da890> False
<keras.layers.pooling.MaxPooling2D object at 0x7fe21c9e0b90> False
<keras.layers.convolutional.Conv2D object at 0x7fe21c9e5b50> False
<keras.layers.convolutional.Conv2D object at 0x7fe21c9f0a10> False
<keras.layers.convolutional.Conv2D object at 0x7fe21c979f90> False
<keras.layers.convolutional.Conv2D object at 0x7fe21c987350> False
<keras.layers.pooling.MaxPooling2D object at 0x7fe21c996b50> False
<keras.layers.convolutional.Conv2D object at 0x7fe21c99d750> False
<keras.layers.convolutional.Conv2D object at 0x7fe21c9a9690> False
<keras.layers.convolutional.Conv2D object at 0x7fe21c9b1e90>

In [4]:
from keras import models
from keras import layers
from keras import optimizers
 
model = models.Sequential()
 
model.add(vgg_conv)
 
model.add(layers.Flatten())
model.add(layers.Dense(1024, activation='relu'))
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(33, activation='softmax'))
 
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg19 (Model)                (None, 7, 7, 512)         20024384  
_________________________________________________________________
flatten_1 (Flatten)          (None, 25088)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 1024)              25691136  
_________________________________________________________________
dense_2 (Dense)              (None, 512)               524800    
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 33)                16929     
Total params: 46,257,249
Trainable params: 28,592,673
Non-trainable params: 17,664,576
_________________________________

In [5]:
train_dir = '/home/asus/Desktop/Dataset_splits/train'
validation_dir = '/home/asus/Desktop/Dataset_splits/validation'

In [6]:
import numpy as np
from random import shuffle

In [7]:
train_datagen = ImageDataGenerator(
      rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
  height_shift_range=0.2,
    horizontal_flip=True,
   fill_mode='nearest'
)
 
validation_datagen = ImageDataGenerator(rescale=1./255)
 
train_batchsize = 64
val_batchsize = 20
 
train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(224, 224),
        batch_size=train_batchsize,
        class_mode='categorical')
 
validation_generator = validation_datagen.flow_from_directory(
        validation_dir,
        target_size=(224, 224),
        batch_size=val_batchsize,
        class_mode='categorical',
        shuffle=True)

Found 22110 images belonging to 33 classes.
Found 4132 images belonging to 33 classes.


In [8]:
print(train_generator.class_indices)

{'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot': 0, 'Corn_(maize)___Common_rust_': 1, 'Corn_(maize)___healthy': 2, 'GD_Alternia': 3, 'GD_Deficiency': 4, 'GD_Healthy': 5, 'GD_Leaf_Blight': 6, 'GD_Rust': 7, 'GD_Spidermites': 8, 'GD_Toxicity': 9, 'GD_Webbloch': 10, 'Paddy_Bacterial_leaf_blight': 11, 'Paddy_Brown_spot': 12, 'Pepper__bell___Bacterial_spot': 13, 'Pepper__bell___healthy': 14, 'Potato___Early_blight': 15, 'Potato___Late_blight': 16, 'Potato___healthy': 17, 'Tomato_Bacterial_spot': 18, 'Tomato_Early_blight': 19, 'Tomato_Late_blight': 20, 'Tomato_Leaf_Mold': 21, 'Tomato_Septoria_leaf_spot': 22, 'Tomato_Spider_mites_Two_spotted_spider_mite': 23, 'Tomato__Target_Spot': 24, 'Tomato__Tomato_YellowLeaf__Curl_Virus': 25, 'Tomato__Tomato_mosaic_virus': 26, 'Tomato_healthy': 27, 'pomo_Alternaria_Alternata': 28, 'pomo_Anthracnose': 29, 'pomo_Bacterial_Blight': 30, 'pomo_Cercospora_LeafSpot': 31, 'pomo_Healthy_Leaves': 32}


In [9]:
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-4),
              metrics=['acc'])

history = model.fit_generator(
      train_generator,
      steps_per_epoch=train_generator.samples/train_generator.batch_size ,
      epochs=10,
      validation_data=validation_generator,
      validation_steps=validation_generator.samples/validation_generator.batch_size,
      verbose=1)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Epoch 1/10
346/345 [==============================] - 4751s 14s/step - loss: 0.0640 - acc: 0.9790 - val_loss: 0.0485 - val_acc: 0.9867
Epoch 2/10
346/345 [==============================] - 3922s 11s/step - loss: 0.0385 - acc: 0.9860 - val_loss: 0.0274 - val_acc: 0.9896
Epoch 3/10
346/345 [==============================] - 4132s 12s/step - loss: 0.0313 - acc: 0.9884 - val_loss: 0.0323 - val_acc: 0.9904
Epoch 4/10
346/345 [==============================] - 4612s 13s/step - loss: 0.0271 - acc: 0.9897 - val_loss: 0.0183 - val_acc: 0.9918
Epoch 5/10
346/345 [==============================] - 3677s 11s/step - loss: 0.0239 - acc: 0.9907 - val_loss: 0.0072 - val_acc: 0.9918
Epoch 6/10
346/345 [==============================] - 4374s 13s/step - loss: 0.0214 - acc: 0.9919 - val_loss: 0.0173 - val_acc: 0.9928
Epoch 7/10
346/345 [==============================] - 4331s 13s/step - loss: 0.0200 - acc: 0.99

In [10]:
model.save('anuvgg19.h5')